In [29]:
__author__ = "me"
__date__ = "2016_02_01"

%pylab inline
import sqlite3
import datetime
import dateutil.parser
import os
import ast

Populating the interactive namespace from numpy and matplotlib


In [35]:
def getFiles(folder, format, full=True, filtr=None):
    for f in os.listdir(folder):
        if f.endswith(format):
            if filtr==None or filtr not in f:
                if full:
                    yield os.path.join(folder, f)
                else:
                    yield f

In [36]:
def getSQLiteTweets(path):
    conn = sqlite3.connect(path)
    df = conn.cursor().execute('SELECT id, timestamp, lon, lat, tweet, user_id, rtwts, fvrts, application, source FROM tweets').fetchall() ## all but raw data
    conn.close()
    return df

In [37]:
tweetE = '''CREATE TABLE IF NOT EXISTS tweets (id INTEGER PRIMARY KEY, 
                                               timestamp INTEGER, 
                                               lon REAL, lat REAL, 
                                               tweet TEXT, 
                                               user_id INTEGER, 
                                               rtwts INTEGER,
                                               fvrts INTEGER,
                                               application TEXT,
                                               source TEXT )'''

In [38]:
def mergeAll(path):
    dbPath = path + 'MERGED_ALL.db'

    conn = sqlite3.connect(dbPath)
    c = conn.cursor()
    c.execute(tweetE)
    
    for f in getFiles(path, 'db', 'MERGED_ALL'):
        print f
        if 'MERGED_ALL' not in f:
            tweets = getSQLiteTweets(f)
            c.executemany('INSERT OR IGNORE INTO tweets VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tweets )
            conn.commit()
    
    conn.close()
    print 'Databases Merged!'
    print dbPath

In [39]:
path = '/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/aggregated/'
mergeAll(path)

/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/aggregated/gnip_twitter.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/aggregated/MERGED_ALL.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/aggregated/MERGED_SCRAPER.db
Databases Merged!
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/aggregated/MERGED_ALL.db
